<a href="https://colab.research.google.com/github/khorneflakes-dev/Proyecto-Final-YELP/blob/main/review_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pandas as pd

In [6]:
df_business_id = pd.read_csv('/content/drive/MyDrive/business_index.csv')
df_business_id.columns = ['id_business', 'business_id']
df_user_id = pd.read_parquet('/content/drive/MyDrive/codes_users.parquet.gzip')

In [7]:
path = '/content/drive/MyDrive/test/Dataset Yelp/review.json'

In [8]:
main_chunk = pd.read_json(path, lines=True, chunksize=400000)


funcion para pasar al chunk

In [9]:
def review_etl(df):
  df.drop(['text'], axis=1, inplace=True)
  df_aux = pd.merge(df, df_business_id, left_on='business_id', right_on='business_id', how='inner')
  df_aux.drop(['business_id'], axis=1, inplace=True)
  df_aux['id_review'] = df_aux.index
  df_aux.drop(['review_id'], axis=1, inplace=True)

  df_aux2 = pd.merge(df_aux, df_user_id, left_on='user_id', right_on='user_id', how='inner')
  df_aux2.drop(['user_id'], axis=1, inplace=True)
  df_aux2.drop(['useful','funny','cool'], axis=1, inplace=True)
  df_aux2 = df_aux2.rename(columns={"n_user_id": "id_user"})
  return df_aux2

In [10]:
chunk_list = []

In [11]:
for chunk in main_chunk:
  chunk_list.append(review_etl(chunk))

In [12]:
df_final = pd.concat(chunk_list)

In [14]:
df_final = df_final[['id_review','id_user','id_business','stars','date']]

In [15]:
df_final.to_parquet('/content/drive/MyDrive/output/reviews2.parquet.gzip', compression='gzip')

In [16]:
df_final.head(5)

,id_review,id_user,id_business,stars,date
0,0,124234,1868,3,2018-07-07 22:09:11
1,87050,124234,10267,5,2018-04-25 17:04:09
2,142907,124234,8317,3,2017-03-05 14:26:14
3,1,151657,1868,2,2017-05-13 17:06:55
4,2,13581,1868,4,2017-08-08 00:58:18


In [17]:
df_final.shape

(6990247, 5)